In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#PostgreSQL access data
host = "localhost"
port = "5432"
database = "domainanalysis"
user = "postgres"
password = "postgres"
table = "domain"

# PostgreSQL connection url
connection = f"jdbc:postgresql://{host}:{port}/{database}"

# Create a Spark session
spark = SparkSession.builder \
    .appName("ETL_DomainAnaylsis") \
    .getOrCreate()

# Read data from the database
domains_df = spark.read \
                .format("jdbc") \
                .option("url", connection) \
                .option("dbtable", table) \
                .option("user", user) \
                .option("password", password) \
                .load()

# Display the data frame
#domains_df.show()

In [ ]:
# Count the values of each column with descending sort and show it
domains_df.groupBy("Top-Level-Domain").count().sort(desc("count")).show(20, False)
domains_df.groupBy("MX-Record").count().sort(desc("count")).show(20, False)
domains_df.groupBy("A-Record").count().sort(desc("count")).show(20, False)

In [ ]:
#
domains_df.withColumn('MX-Record_Single_Count', explode(split(col('MX-Record'), ','))) \
        .groupBy('MX-Record_Single_Count') \
        .count() \
        .sort(desc("count")) \
        .show(20, False)


domains_df.withColumn('A-Record_Single_Count', explode(split(col('A-Record'), ','))) \
        .groupBy('A-Record_Single_Count') \
        .count() \
        .sort(desc("count")) \
        .show(20, False)



In [ ]:
import dns
from dns import resolver

def getARecords(domain):
    try:
        result = dns.resolver.resolve("www." + domain, 'A')
        for ipval in result:
            return ipval.to_text()
    except dns.resolver.NXDOMAIN:
        return "Domain not available"
    except dns.resolver.NoAnswer:
        return "No Answer"
    except dns.resolver.NoNameservers:
        return "No Nameservers"
    except dns.resolver.Timeout:
        return "Timeout"


def getMXRecords(domain):
    try:
        result = dns.resolver.resolve(domain, 'MX')
        for mail in result:
            return mail.exchange.to_text()
    except dns.resolver.NXDOMAIN:
        return "Domain not available"
    except dns.resolver.NoAnswer:
        return "No Answer"
    except dns.resolver.NoNameservers:
        return "No Nameservers"

In [ ]:
udf_getARecords = udf(getARecords, StringType())
udf_getMXRecords = udf(getMXRecords, StringType())

In [ ]:
domains_df = domains_df.withColumn("A-Record_Checked", lit(udf_getARecords("Top-Level-Domain")))
domains_df = domains_df.withColumn("MX-Record_Checked", lit(udf_getMXRecords("Top-Level-Domain")))

In [ ]:
domains_df.show(20)